In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer

In [ ]:
# Definir el dispositivo (CPU o MPS)
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

In [ ]:
# Cargar los datos preprocesados
train_data = pd.read_csv('/Users/rosannabm/Desktop/Inteligencia-Artificial/Proyecto Final/stanfordSentimentTreebank/train_data.csv')
val_data = pd.read_csv('/Users/rosannabm/Desktop/Inteligencia-Artificial/Proyecto Final/stanfordSentimentTreebank/val_data.csv')

In [ ]:
# Tokenización
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class SST2Dataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
# Preparar el dataset
train_texts = train_data['sentence'].tolist()
train_labels = train_data['label'].tolist()
val_texts = val_data['sentence'].tolist()
val_labels = val_data['label'].tolist()

train_dataset = SST2Dataset(train_texts, train_labels, tokenizer, max_len=128)
val_dataset = SST2Dataset(val_texts, val_labels, tokenizer, max_len=128)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [7]:
# Definición del modelo LSTM
import torch.nn as nn
from transformers import BertModel

class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super(LSTMClassifier, self).__init__()
        self.embedding = BertModel.from_pretrained('bert-base-uncased')
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            embedded = self.embedding(input_ids, attention_mask=attention_mask)[0]
        lstm_output, (hidden, cell) = self.lstm(embedded)
        if self.lstm.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        return self.fc(hidden)

In [ ]:
# Entrenamiento y evaluación del modelo
import torch.optim as optim
import numpy as np

def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0

    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels = d["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, labels)

        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / n_examples, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, labels)

            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

    return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
# Inicializar el modelo, pérdida y optimizador
model = LSTMClassifier(embedding_dim=768, hidden_dim=256, output_dim=2, n_layers=2, bidirectional=True, dropout=0.3)
model = model.to(device)

loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * 10

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

In [ ]:
# Entrenamiento
for epoch in range(10):
    train_acc, train_loss = train_epoch(model, train_loader, loss_fn, optimizer, device, scheduler, len(train_dataset))
    print(f'Epoch {epoch+1}/{10} - Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}')
    
    val_acc, val_loss = eval_model(model, val_loader, loss_fn, device, len(val_dataset))
    print(f'Epoch {epoch+1}/{10} - Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}')